# Discussion

This notebook discusses the paper titled **"Regularizing and Optimizing LSTM Language Models"** by *Merity et al* that was published in 2017.  

### Introduction

Recurrent Neural Networks and their variations are very likely to overfit the training data. This is due to the large network formed by unfolding each cell of the RNN, and *relatively* small number of parameters (since they are shared over each time step) and training data. Thus, the perplexities obtained on the test data are often quite larger than expected. Several attempts have been made to minimize this problem using varied **regularization** techniques. This paper tackles this issue by proposing a model that combines several of such existing methods.

*Merity et al*'s model is a modification of the standard **LSTM** in which *DropConnect* is applied to the hidden weights in the *recurrent* connections of the LSTM for regularization. The dropout mask for each weight is preserved and the same mask is used across all time steps, thereby adding negligible computation overhead. Apart from this, several other techniques have been incorporated :
* **Variational dropout** : 
* **Embedding dropout** :
* **AR and TAR** : AR (Activation Regularization) and TAR (Temporal Activation Regularization)
* **Weight tying** :
* **Variable backpropagation steps** :
* **Independent sizes of word embeddings and hidden layer** :

The paper also introduces a new optimization algorithm, namely **Non-monotonically Triggered Averaged Stochastic Gradient Descent** or NT-ASGD, which can be programmatically described as follows :

In [ ]:
def NT_ASGD(args):
    . . .
    return averaged_weights